# Dusty box

This notebook contains analysis of the dusty box test for multigrain dust.

In [ ]:
import pathlib
import sys

In [ ]:
%matplotlib widget
import plonk

In [ ]:
sys.path.insert(0, '../modules')
from multigrain import dustybox

## Time evolution

In [ ]:
root_directory = pathlib.Path('~/runs/multigrain/dustybox/time_evolution').expanduser()
_paths = sorted(list(root_directory.glob('*')))
paths = {p.name: p for p in _paths}
paths

### Calculate velocity differential time evolution

In [ ]:
data = dict()
exact1 = dict()
exact2 = dict()

for name, path in paths.items():
    try:
        print(f'Running analysis for {name}...')
        sim = plonk.load_sim(prefix='dustybox', directory=path)
        data[name] = dustybox.calculate_differential_velocity(sim)
        exact1[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=True)
        exact2[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=False)
    except:
        print(f'Failed to load {name}')

### Plot time evolution

In [ ]:
fig = dustybox.plot_differential_velocity_all(data, exact1, exact2, figsize=(10, 6))

### Calculate velocity differential error

In [ ]:
error = dict()

for name, path in paths.items():
    print(f'Calculating error for {name}...')
    sim = plonk.load_sim(prefix='dustybox', directory=path)
    error[name] = dustybox.calculate_error(sim)

### Plot error

In [ ]:
fig = dustybox.plot_error_all(error, figsize=(10, 6))

## Stability

In [ ]:
# Parameter in Phantom patch: "comparing" dt_force and dt_drag
DTFORCE_TO_DTDRAG = 3.6

In [ ]:
root_directory = pathlib.Path('~/runs/multigrain/dustybox/stability').expanduser()
_paths = sorted(list(root_directory.glob('*')))
paths = {p.name: p for p in _paths}
paths

In [ ]:
dust_to_gas = sorted(list(set([float(key[4:8]) for key in paths.keys()])))
C_force = sorted(list(set([float(key[17:]) for key in paths.keys()])))
dtdrag_fac = [val * DTFORCE_TO_DTDRAG for val in C_force]

### Calculate velocity differential time evolution

In [ ]:
data = dict()
exact1 = dict()
exact2 = dict()

for name, path in paths.items():
    try:
        print(f'Running analysis for {name}...')
        sim = plonk.load_sim(prefix='dustybox', directory=path)
        data[name] = dustybox.calculate_differential_velocity(sim)
        exact1[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=True)
        exact2[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=False)
    except:
        print(f'Failed to load {name}')

### Plot stability

In [ ]:
n_dust_to_gas = len(dust_to_gas)
fig = dustybox.plot_differential_velocity_all(data, exact1, exact2, ncols=4, figsize=(12, 10), transpose=True)
for ax, eps in zip(fig.axes[:n_dust_to_gas], dust_to_gas):
    ax.set_title(rf'$\epsilon = {eps:.2f}$')
for ax, fac in zip(fig.axes[::n_dust_to_gas], dtdrag_fac):
    ax.set_ylabel(rf'{fac:.4f}')